In [2]:
from tensorflow.keras.layers import Bidirectional,Concatenate,Permute,Dot,Input,LSTM,Multiply
from tensorflow.keras.layers import RepeatVector,Dense,Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Preparing Dataset 
Here we will synthetically generate a dataset and write helper functions for the same.

In [16]:
# These are formats for generating the date the short , medium and full formats are formats specified by babel.dates import , check babel docs
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']


def load_date() -> tuple:
    fake = Faker()
    date = fake.date_object()

    try:
        human_readable = format_date(date,random.choice(FORMATS),locale="en_US")
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(",","")
        machine_readable = date.isoformat()
    
    except AttributeError as e:
        return None,None,None

    return human_readable,machine_readable ,date


def load_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30 # This is the max length of the date string we assume 

    for _ in tqdm(range(m)):
        h,m,_ = load_date()
        if h is not None:
            dataset.append((h,m))
            human_vocab.update(h)
            machine_vocab.update(m)

    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v:k for k,v in inv_machine.items()}

    return dataset,human,machine,inv_machine

In [17]:
dataset, human, machine, inv_machine = load_dataset(10000)

100%|██████████| 10000/10000 [01:03<00:00, 158.11it/s]


In [18]:
dataset[:10]

[('mar 9 2007', '2007-03-09'),
 ('february 11 1972', '1972-02-11'),
 ('2 nov 2019', '2019-11-02'),
 ('tuesday may 4 1999', '1999-05-04'),
 ('thursday december 30 2004', '2004-12-30'),
 ('friday november 10 2017', '2017-11-10'),
 ('wednesday march 8 1995', '1995-03-08'),
 ('2 april 1988', '1988-04-02'),
 ('monday march 25 2019', '2019-03-25'),
 ('24.03.22', '2022-03-24')]

In [ ]:
def preprocess_data(dataset,human,machine,Tx,Ty):
    